In [1]:
import sys
!{sys.executable} -m pip install --upgrade joblib

In [2]:
import os
os.makedirs('code', exist_ok=True)

In [18]:
%%writefile code/score.py
import os
import torch
import torch.nn as nn
import joblib
import numpy as np
import json

# Features in the same order as training
FEATURE_ORDER = [
    "all_util", "open_acc_6m", "open_rv_12m", "il_util",
    "delinq_2yrs", "acc_now_delinq", "total_bal_il",
    "last_fico_range_low", "last_fico_range_high", "loan_amnt"
]

class ANNModule(nn.Module):
    def __init__(self, input_dim=10, hidden_layers=[128, 64, 32], dropout=0.3):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for h in hidden_layers:
            layers.append(nn.Linear(prev_dim, h))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            prev_dim = h
        layers.append(nn.Linear(prev_dim, 1))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x).view(-1)


def init():
    global model, scaler, device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model_dir = os.environ.get("AZUREML_MODEL_DIR", "./model")
    model_path = os.path.join(model_dir, "model", "best_ann_model.pth")
    scaler_path = os.path.join(model_dir, "model", "scaler.pkl")

    # Load scaler
    scaler = joblib.load(scaler_path)

    # Load model
    model = ANNModule()
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()


def run(raw_data):
    import logging
    logger = logging.getLogger(__name__)
    
    try:
        # If Azure sends raw JSON string, parse it
        if isinstance(raw_data, str):
            raw_data = json.loads(raw_data)

        logger.info(f"Received data: {raw_data}")

        # If it's a dict with "data" key, extract
        if isinstance(raw_data, dict) and "data" in raw_data:
            raw_data = raw_data["data"]

        # Now raw_data should be a list of dicts
        data_list = []
        for d in raw_data:
            if not isinstance(d, dict):
                raise ValueError(f"Expected dict, got {type(d)}: {d}")
            row = [d.get(feat, None) for feat in FEATURE_ORDER]
            if None in row:
                missing = [FEATURE_ORDER[i] for i, v in enumerate(row) if v is None]
                raise ValueError(f"Missing features: {missing}")
            data_list.append(row)

        # Convert to numpy array, scale, and run model
        data = np.array(data_list, dtype=np.float32)
        data_scaled = scaler.transform(data)
        inputs = torch.tensor(data_scaled, dtype=torch.float32).to(device)

        with torch.no_grad():
            logits = model(inputs)
            probs = torch.sigmoid(logits)
            preds = (probs > 0.5).int()

        logger.info(f"Predictions: {preds.cpu().numpy()}")
        return {
            "predictions": preds.cpu().numpy().tolist(),
            "probabilities": probs.cpu().numpy().tolist()
        }

    except Exception as e:
        logger.error(f"Exception in run: {type(e)}: {e}", exc_info=True)
        return {"error": f"{type(e)}: {e}"}

Overwriting code/score.py


In [1]:
import sys
!{sys.executable} -m pip install --upgrade azure-ai-ml azure-identity pandas numpy torch

  Using cached azure_identity-1.24.0-py3-none-any.whl.metadata (86 kB)
Using cached azure_identity-1.24.0-py3-none-any.whl (187 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 21.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 24.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.0/888.0 MB 12.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 18.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 26.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 25.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 18.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 25.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.5 MB/s eta 0:00:

In [19]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model, ManagedOnlineEndpoint, ManagedOnlineDeployment, Environment
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())


Found the config file in: ./config.json
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [3]:
model = Model(
    path="model",  # your .pth file
    name="loan_ann_model",
    description="ANN model for loan default prediction",
    type="custom_model",  # <-- NOT mlflow_model
)

registered_model = ml_client.models.create_or_update(model)
print("Registered model:", registered_model.name, registered_model.version)

Uploading model (0.05 MBs): 100%|██████████| 52166/52166 [00:00<00:00, 1671339.46it/s]




Registered model: loan_ann_model 1


In [4]:
%%writefile conda_env.yml

name: pytorch-env
channels:
  - defaults
dependencies:
  - python=3.10
  - pip
  - pip:
      - torch
      - scikit-learn
      - joblib
      - pandas
      - numpy
      - azureml-defaults
      - azureml-inference-server-http

Writing conda_env.yml


In [5]:
env = Environment(
    name="pytorch-env",
    version="4",
    description="PyTorch + dependencies",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    conda_file="conda_env.yml"  # create a YAML with torch, scikit-learn, joblib
)
ml_client.environments.create_or_update(env)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'pytorch-env', 'description': 'PyTorch + dependencies', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/ef6f0ccd-be4a-4f41-8c9f-abc3cf22df34/resourceGroups/mlrg-second/providers/Microsoft.MachineLearningServices/workspaces/ws-uk-ml/environments/pytorch-env/versions/4', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/flyhighpraveen03101/code/Users/flyhighpraveen0310', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7dfee23b73a0>, 'serialize': <msrest.serialization.Serializer object at 0x7dfee23b73d0>, 'version': '4', 'conda_file': {'channels': ['defaults'], 'dependencies': ['python=3.10', 'pip', {'pip

In [8]:
endpoint_name = "loan-ann-endpoint"
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    auth_mode="key"
)
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.1.1) and mlflow-skinny (2.22.1) are different. This may lead to unexpected behavior. Please install the same version of both packages.
  mlflow.mismatch._check_version_mismatch()


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://loan-ann-endpoint.ukwest.inference.ml.azure.com/score', 'openapi_uri': 'https://loan-ann-endpoint.ukwest.inference.ml.azure.com/swagger.json', 'name': 'loan-ann-endpoint', 'description': None, 'tags': {}, 'properties': {'createdBy': 'Praveenkumar Palaniappan', 'createdAt': '2025-08-30T08:16:32.413529+0000', 'lastModifiedAt': '2025-08-30T08:16:32.413529+0000', 'azureml.onlineendpointid': '/subscriptions/ef6f0ccd-be4a-4f41-8c9f-abc3cf22df34/resourcegroups/mlrg-second/providers/microsoft.machinelearningservices/workspaces/ws-uk-ml/onlineendpoints/loan-ann-endpoint', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ef6f0ccd-be4a-4f41-8c9f-abc3cf22df34/providers/Microsoft.MachineLearningServices/locations/ukwest/mfeOperationsStatus/oeidp:0c8abcec-cb75-416c-bd29-0fa02e3118bd:4ba120db-8146-4f14-9194-72492f517d89?api-version=2022-02-01-preview'}, 'print_as_

In [20]:
from azure.ai.ml.entities import CodeConfiguration

registered_model = ml_client.models.get(name="loan_ann_model", label="latest")

deployment = ManagedOnlineDeployment(
    name="loan-fifth-ann-deployment",
    endpoint_name="loan-ann-endpoint",
    model=registered_model.id,
    environment="pytorch-env:4",
    code_configuration=CodeConfiguration(
        code="./code",              # folder containing scoring script
        scoring_script="score.py"         # scoring script filename
    ),
    instance_type="STANDARD_D2AS_V4",
    instance_count=1
)

poller = ml_client.online_deployments.begin_create_or_update(deployment)
print("Deployment started. Polling status...")
status = poller.status()
print(status)

Check: endpoint loan-ann-endpoint exists
Uploading code (0.0 MBs): 100%|██████████| 3348/3348 [00:01<00:00, 2884.77it/s]




Deployment started. Polling status...
InProgress
.................................................................................................

In [13]:
status = poller.status()
print(status)

Updating


In [17]:
ml_client.online_deployments.begin_delete(
    name="loan-third-ann-deployment",   # or "default" depending on what you used
    endpoint_name="loan-ann-endpoint"
).result()